In [ ]:
%run includes/shared-setup.ipynb

In [ ]:

# Load DataFrame from pickle
resources_df = pd.read_pickle('/tmp/resources_df.pkl')

with open('/tmp/all_projects.pkl', 'rb') as f:
    all_projects = pickle.load(f)

## Contact Center AI Platform - Contact Centers

In [ ]:

# google.contactcenteraiplatform
queries = [
    f"""
    SELECT *, '{project}' as project
    FROM google.contactcenteraiplatform.locations
    WHERE projectsId = '{project}'
    """
    for project in all_projects
]
locations_df = run_stackql_queries(queries)
locations = locations_df['locationId'].unique().tolist()
enabled_projects = locations_df['project'].unique().tolist()

queries = [
    f"""
    SELECT 
    displayName
    ,SPLIT_PART(name, '/', 6) as name
    ,'{project}' as project
    ,'{location}' as location
    ,state
    ,customerDomainPrefix
    ,JSON_EXTRACT(adminUser, '$.givenName') || ' ' || JSON_EXTRACT(adminUser, '$.familyName') as adminUser
    ,userEmail
    ,JSON_EXTRACT(instanceConfig, '$.instanceSize') as chatBotUri
    ,JSON_EXTRACT(uris, '$.chatBotUri') as chatBotUri
    ,JSON_EXTRACT(uris, '$.mediaUri') as mediaUri
    ,kmsKey
    FROM google.contactcenteraiplatform.contact_centers
    WHERE projectsId = '{project}' AND locationsId = '{location}'
    """
    for project in enabled_projects
    for location in locations
]
_df = run_stackql_queries(queries)
_df

## Artifact Registry Respositories

In [ ]:

# google.artifactregistry
queries = [
    f"""
    SELECT *, '{project}' as project
    FROM google.artifactregistry.locations
    WHERE projectsId = '{project}'
    """
    for project in artifactregistry_projects
]
locations_df = run_stackql_queries(queries)
locations = locations_df['locationId'].unique().tolist()
enabled_projects = locations_df['project'].unique().tolist()

queries = [
    f"""
    SELECT 
    SPLIT_PART(name, '/', 6) as name
    ,'{project}' as project
    ,'{location}' as location
    ,mode
    ,(sizeBytes/1048576.0) as sizeMB
    ,createTime
    FROM google.artifactregistry.repositories
    WHERE projectsId = '{project}' AND locationsId = '{location}'
    """
    for project in artifactregistry_projects
    for location in locations
]
_df = run_stackql_queries(queries)
_df

## Log Sinks

In [ ]:

# log sinks
folder_rows = resources_df[resources_df['resType'] == 'folder']
foldersIds = folder_rows['name'].str.split('/').str[-1].tolist()

project_rows = resources_df[resources_df['resType'] == 'folder']
projectsIds = project_rows['displayName'].tolist()

org_id = '421482594427'

# org log sink
queries = [
    f"""
    SELECT 
    name
    ,'organization' as level
    ,'{org_id}' as id
    ,description
    ,destination
    ,writerIdentity
    ,includeChildren
    FROM google.logging.sinks
    WHERE organizationsId = '{org_id}' and writerIdentity is not null
    """
]
org_log_sinks_df = run_stackql_queries(queries)

# folder log sinks
queries = [
    f"""
    SELECT 
    name
    ,'folder' as level
    ,'{foldersId}' as id
    ,description
    ,destination
    ,writerIdentity
    ,includeChildren
    FROM google.logging.sinks
    WHERE foldersId = '{foldersId}' and writerIdentity is not null
    """
    for foldersId in foldersIds
]
folder_log_sinks_df = run_stackql_queries(queries)

# project log sinks
queries = [
    f"""
    SELECT 
    name
    ,'project' as level
    ,'{projectsId}' as id
    ,description
    ,destination
    ,writerIdentity
    ,includeChildren
    FROM google.logging.sinks
    WHERE projectsId = '{projectsId}' and writerIdentity is not null
    """
    for projectsId in projectsIds
]
project_log_sinks_df = run_stackql_queries(queries)

pd.concat([org_log_sinks_df, folder_log_sinks_df, project_log_sinks_df], ignore_index=True)